In [3]:
# Package for loading data
import pandas as pd
import numpy as np

# Deep learning package
import tensorflow as tf
import tensorflow.keras as keras

# Metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

# Sampling
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# Train and test split 
from sklearn.model_selection import train_test_split


In [4]:
df = pd.read_csv('../data/processed/preprocessed_data.csv')
df.head()

,Unnamed: 0,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,0,-0.326221,0,0,42,2,-1.225848,1,1,1,0.021886,1
1,1,-0.440036,2,0,41,2,0.117350,1,0,1,0.216534,0
2,2,-1.536794,0,0,42,2,1.333053,3,1,0,0.240687,1
3,3,0.501521,0,0,39,2,-1.225848,2,0,0,-0.108918,0
4,4,2.063884,2,0,43,2,0.785728,1,1,1,-0.365276,0


In [5]:
# Drop the column Unnamed: 0
df.drop(["Unnamed: 0"],axis=1,inplace=True)

In [6]:
# Independents features
X = df.drop(['churn'],axis=1)

# Target feature
y = df.churn

In [7]:
# Train and test split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
# Sampling
sampler = RandomOverSampler()
X_ru , y_ru = sampler.fit_resample(X_train,y_train)

In [9]:
keras.backend.clear_session()

# Build the architecture of the model
model = keras.models.Sequential()
model.add(keras.layers.Dense(units=16,activation='relu',kernel_initializer='uniform',input_dim=X.shape[1]))
model.add(keras.layers.Dense(units=32,activation='relu'))
model.add(keras.layers.Dense(units=32,activation='relu'))
model.add(keras.layers.Dense(units=64,activation='relu'))
model.add(keras.layers.Dense(units=64,activation='relu'))
model.add(keras.layers.Dense(units=1,activation='sigmoid',kernel_initializer='uniform'))


In [10]:
# Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])

In [11]:
# Fit the model
model.fit(X_ru,y_ru,epochs=50,batch_size=8)

Epoch 1/50
1589/1589 [==============================] - 2s 1ms/step - loss: 0.6121 - accuracy: 0.6691
Epoch 2/50
1589/1589 [==============================] - 1s 908us/step - loss: 0.5904 - accuracy: 0.6919
Epoch 3/50
1589/1589 [==============================] - 2s 1ms/step - loss: 0.5863 - accuracy: 0.6963
Epoch 4/50
1589/1589 [==============================] - 2s 1ms/step - loss: 0.5740 - accuracy: 0.7076
Epoch 5/50
1589/1589 [==============================] - 2s 1ms/step - loss: 0.5639 - accuracy: 0.7094
Epoch 6/50
1589/1589 [==============================] - 2s 1ms/step - loss: 0.5422 - accuracy: 0.7252
Epoch 7/50
1589/1589 [==============================] - 2s 1ms/step - loss: 0.5045 - accuracy: 0.7491
Epoch 8/50
1589/1589 [==============================] - 2s 1ms/step - loss: 0.4852 - accuracy: 0.7587
Epoch 9/50
1589/1589 [==============================] - 1s 917us/step - loss: 0.4804 - accuracy: 0.7643
Epoch 10/50
1589/1589 [==============================] - 2s 965us/step - loss:

In [12]:
model.evaluate(X_test,y_test)

63/63 [==============================] - 0s 847us/step - loss: 0.4720 - accuracy: 0.7795


[0.47199657559394836, 0.7795000076293945]

In [13]:
y_pred = model.predict(X_test)
y_pred[:5]

array([[0.07618281],
       [0.04713839],
       [0.06238341],
       [0.53705275],
       [0.07451606]], dtype=float32)

In [47]:
y_pred_new = []

for element in y_pred:
	if element > 0.56:
		y_pred_new.append(1)
	else:
		y_pred_new.append(0)

In [48]:
print(classification_report(y_test,y_pred_new))

              precision    recall  f1-score   support

           0       0.93      0.82      0.87      1607
           1       0.51      0.75      0.60       393

    accuracy                           0.81      2000
   macro avg       0.72      0.79      0.74      2000
weighted avg       0.85      0.81      0.82      2000



In [105]:
# Save the model
model.save('../models/Deep_Learning_model')

INFO:tensorflow:Assets written to: ../models/Deep_Learning_model/assets
